In [33]:
from env import config, BASE_DIR
from sqlalchemy import create_engine, text

In [34]:
DATABASE_URL = config("FLIGHT_PRICES_DATABASE_URL", cast=str, default=None)
DATABASE_URL is not None

True

In [35]:
engine = create_engine(str(DATABASE_URL))

In [36]:
DATASET_DIR = BASE_DIR / "datasets"
DATASET_DIR.exists()

True

In [37]:
FLIGHT_PRICES_CSV = DATASET_DIR / "itineraries-min-50k.csv"
FLIGHT_PRICES_CSV.exists()

True

In [38]:
import pandas as pd

In [39]:
df = pd.read_csv(FLIGHT_PRICES_CSV)
df.head(n=15)

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H29M,0.0,False,False,...,1650223560,2022-04-17T15:26:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,8940,947,coach
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0.0,False,False,...,1650200400,2022-04-17T09:00:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9000,947,coach
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0.0,False,False,...,1650218700,2022-04-17T14:05:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757-200,9000,947,coach
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H32M,0.0,False,False,...,1650227460,2022-04-17T16:31:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9120,947,coach
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H34M,0.0,False,False,...,1650213180,2022-04-17T12:33:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9240,947,coach
5,79eda9f841e226a1e2121d74211e595c,2022-04-16,2022-04-17,ATL,BOS,VH0AUEL1,PT2H38M,0.0,False,False,...,1650216180,2022-04-17T13:23:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9480,947,coach
6,9335fae376c38bb61263281779f469ec,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT4H12M,0.0,False,False,...,1650203400||1650213120,2022-04-17T09:50:00.000-04:00||2022-04-17T12:3...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A320||Airbus A320,5400||7500,228||728,coach||coach
7,3904bf87f2d1daf334f1ae7e3b876028,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT5H18M,0.0,False,False,...,1650203400||1650217080,2022-04-17T09:50:00.000-04:00||2022-04-17T13:3...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A320||Boeing 737-800,5400||8280,228||728,coach||coach
8,d93988734c44a3c075d9efe373352507,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT5H32M,0.0,False,False,...,1650198060||1650213120,2022-04-17T08:21:00.000-04:00||2022-04-17T12:3...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A319||Airbus A320,4860||7500,228||728,coach||coach
9,562e7d5dd6ecbf1509c0c19711dbdca9,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT6H38M,0.0,False,False,...,1650198060||1650217080,2022-04-17T08:21:00.000-04:00||2022-04-17T13:3...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A319||Boeing 737-800,4860||8280,228||728,coach||coach


In [40]:
df.dtypes

legId                                 object
searchDate                            object
flightDate                            object
startingAirport                       object
destinationAirport                    object
fareBasisCode                         object
travelDuration                        object
elapsedDays                          float64
isBasicEconomy                          bool
isRefundable                            bool
isNonStop                               bool
baseFare                             float64
totalFare                            float64
seatsRemaining                       float64
totalTravelDistance                  float64
segmentsDepartureTimeEpochSeconds     object
segmentsDepartureTimeRaw              object
segmentsArrivalTimeEpochSeconds       object
segmentsArrivalTimeRaw                object
segmentsArrivalAirportCode            object
segmentsDepartureAirportCode          object
segmentsAirlineName                   object
segmentsAi

In [41]:
df['flightDate']=pd.to_datetime(df['flightDate'])

In [48]:
group_by_columns = ['flightDate' , 'startingAirport' , 'destinationAirport', 'isBasicEconomy', 'isRefundable', 'isNonStop', 'segmentsAirlineName']
target_column = 'totalFare'
desired_columns = group_by_columns + [target_column]
sub_df = df.copy()[desired_columns]
sub_df = sub_df.groupby(group_by_columns)[target_column].mean().reset_index()
sub_df['startingAirport'] = sub_df['startingAirport'].apply(lambda x: f"{x}".strip())
sub_df['destinationAirport'] = sub_df['destinationAirport'].apply(lambda x: f"{x}".strip())
sub_df['totalFare'] = sub_df['totalFare'].round(0).apply(lambda x: int(x))
sub_df.dropna(inplace=True)
sub_df.head(n=40)

,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,segmentsAirlineName,totalFare
0,2022-04-17,ATL,BOS,False,False,False,American Airlines||American Airlines,273
1,2022-04-17,ATL,BOS,False,False,False,Spirit Airlines||Spirit Airlines,301
2,2022-04-17,ATL,BOS,False,False,False,United||United,352
3,2022-04-17,ATL,BOS,False,False,True,American Airlines,355
4,2022-04-17,ATL,BOS,False,False,True,Delta,307
5,2022-04-17,ATL,BOS,False,False,True,JetBlue Airways,302
6,2022-04-17,ATL,BOS,True,False,False,United||United,307
7,2022-04-17,ATL,CLT,False,False,False,American Airlines||American Airlines,492
8,2022-04-17,ATL,CLT,False,False,False,JetBlue Airways||JetBlue Airways,543
9,2022-04-17,ATL,CLT,False,False,False,United||United,467


In [49]:
sub_df.dtypes

flightDate             datetime64[ns]
startingAirport                object
destinationAirport             object
isBasicEconomy                   bool
isRefundable                     bool
isNonStop                        bool
segmentsAirlineName            object
totalFare                       int64
dtype: object

In [50]:
out_df = sub_df.copy()

In [51]:
out_df.to_sql("flight_prices", engine, if_exists='replace', index=True)

9592

In [52]:
rows , cols = out_df.shape
rows , cols

(9592, 8)

In [54]:
with engine.connect() as conn:
   results = conn.execute(text(f"DESCRIBE flight_prices"))
   print(results.all())

[('index', 'bigint(20)', 'YES', 'MUL', None, ''), ('flightDate', 'datetime', 'YES', '', None, ''), ('startingAirport', 'text', 'YES', '', None, ''), ('destinationAirport', 'text', 'YES', '', None, ''), ('isBasicEconomy', 'tinyint(1)', 'YES', '', None, ''), ('isRefundable', 'tinyint(1)', 'YES', '', None, ''), ('isNonStop', 'tinyint(1)', 'YES', '', None, ''), ('segmentsAirlineName', 'text', 'YES', '', None, ''), ('totalFare', 'bigint(20)', 'YES', '', None, '')]
